In [10]:
import pandas as pd
import numpy as np
#from preprocess import preprocess_text
#from preprocess import clean_df
from sklearn.preprocessing import LabelEncoder

from model_class import Model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.linear_model import SGDClassifier

In [11]:
data = pd.read_csv("labelled_newscatcher_dataset.csv", sep=";")
data

#X, y = clean_df(data)

,topic,link,domain,published_date,title,lang
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en
...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en


In [12]:
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter


def preprocess_text(text):

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    
    # Tokenization
    tokens = word_tokenize(text)

    # Lemmatization using WordNetLemmatizer
    words = [lemmatizer.lemmatize(word) for word in tokens]
    text = ' '.join(words)

    return text
  

def clean_df(text):
    text = text[["topic","title"]]
    text["title"] = text["title"].astype(str)
    text = text[text["title"] != "nan"].reset_index(drop=True)
    most_common = freq_rare_words(text["title"])
    stop_words = set(stopwords.words('english'))
    
    most_common_wo_sw  = [word for word in most_common if word[0] not in stop_words]
    print(most_common_wo_sw[:100])

    RAREWORDS = [w for (w, word_count) in most_common_wo_sw[-30:]]
    FREQWORDS = [w for (w, word_count) in most_common_wo_sw[:30]]

    for i in range(len(text)):
        text.loc[i,"title"] = remove_FreqRare_words(text.loc[i,"title"], RAREWORDS, FREQWORDS, stop_words)
        text.loc[i,"title"] = preprocess_text(text.loc[i,"title"])

    target = text["topic"]
    text = text["title"]
    return (text,target)

def freq_rare_words(text):
    full_text = ' '.join(text)
    split_text = full_text.split()
    lowercase_words = [word.lower() for word in split_text]
    count = Counter(lowercase_words)
    most_common = count.most_common()
    
    return most_common

def remove_FreqRare_words(text, RAREWORDS, FREQWORDS, stop_words):
    
    # Tokenization
    tokens = word_tokenize(text)

    filtered_words = [ word for word in tokens if (word not in RAREWORDS) and (word not in FREQWORDS) and (word not in stop_words) and word.isalnum()]

    filtered_text = ' '.join(filtered_words)
    return filtered_text

In [13]:

filtered_X, y = clean_df(data)  



C:\Users\quent\AppData\Local\Temp\ipykernel_24364\3894934197.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



[('covid-19', 8897), ('new', 8442), ('coronavirus', 5755), ('-', 4962), ('says', 3627), ('2020', 3371), ('market', 3310), ('cases', 3124), ('|', 2571), ('–', 2521), ('us', 2446), ('first', 2164), ('global', 2090), ('could', 1881), ('man', 1875), ('news', 1781), ('vaccine', 1767), ('&', 1547), ('covid', 1543), ('may', 1517), ('trump', 1444), ('study', 1443), ('virus', 1395), ('health', 1367), ('coronavirus:', 1338), ('august', 1311), ('top', 1273), ('death', 1272), ('report', 1267), ('live', 1261), ('one', 1258), ('reports', 1257), ('back', 1246), ('million', 1223), ('get', 1217), ('star', 1191), ('day', 1184), ('two', 1182), ('season', 1164), ('apple', 1152), ('pandemic', 1147), ('uk', 1133), ('league', 1101), ('reveals', 1062), ('transfer', 1062), ('amid', 1060), ('5', 1058), ('google', 1054), ('people', 1049), ('test', 1046), ('second', 1018), ('2', 1007), ('lockdown', 1003), ('3', 995), ('china', 988), ('game', 986), ('police', 977), ('watch', 973), ('deal', 964), ('forecast', 953),

In [14]:
unique_x = list(set(' '.join(filtered_X).split()))
print(len(unique_x))

65453


In [15]:
print((freq_rare_words(data["title"]))[:100])

[('to', 31897), ('in', 23742), ('of', 19144), ('the', 18780), ('for', 16085), ('and', 15016), ('on', 11247), ('a', 10508), ('covid-19', 8897), ('with', 8544), ('new', 8442), ('as', 7777), ('is', 6537), ('after', 6024), ('coronavirus', 5755), ('at', 5351), ('-', 4962), ('from', 4927), ('by', 4639), ('be', 3639), ('says', 3627), ('2020', 3371), ('market', 3310), ('over', 3273), ('cases', 3124), ('will', 2886), ('how', 2857), ('more', 2845), ('|', 2571), ('–', 2521), ('us', 2446), ('are', 2376), ('out', 2191), ('first', 2164), ('you', 2134), ('global', 2090), ('up', 2061), ('not', 2051), ('has', 2016), ('it', 1948), ('this', 1944), ('about', 1938), ('could', 1881), ('man', 1875), ('have', 1837), ('news', 1781), ('vaccine', 1767), ('his', 1747), ('what', 1682), ('who', 1678), ('her', 1612), ('can', 1593), ('that', 1579), ('&', 1547), ('covid', 1543), ('may', 1517), ('why', 1444), ('trump', 1444), ('study', 1443), ('but', 1430), ('virus', 1395), ('into', 1393), ('health', 1367), ('coronavir

In [16]:
class_labels = ['SCIENCE', 'TECHNOLOGY', 'HEALTH', 'WORLD', 'ENTERTAINMENT', 'SPORTS', 'BUSINESS', 'NATION']

label_encoder = LabelEncoder() # instantiate a label encoder 
y_enc = label_encoder.fit_transform(y)

In [17]:
Mnb_Count_model = Model(filtered_X,y_enc, MultinomialNB(),CountVectorizer, class_labels) # Instanciate the model class
Mnb_Count_model.fit() # Fitting the model
y_pred = Mnb_Count_model.predict() # Make the predictions
class_labels=label_encoder.inverse_transform(range(8))

Mnb_Count_model.report(y_pred,class_labels) # Show report on model performances

               precision    recall  f1-score   support

     BUSINESS       0.80      0.73      0.76      2922
ENTERTAINMENT       0.83      0.87      0.85      2952
       HEALTH       0.71      0.84      0.77      3006
       NATION       0.70      0.67      0.68      2970
      SCIENCE       0.89      0.74      0.81       764
       SPORTS       0.94      0.91      0.93      3009
   TECHNOLOGY       0.85      0.86      0.86      3111
        WORLD       0.68      0.65      0.67      3021

     accuracy                           0.79     21755
    macro avg       0.80      0.78      0.79     21755
 weighted avg       0.79      0.79      0.79     21755



In [18]:
LR_Tfidf_model = Model(filtered_X,y_enc, LogisticRegression(),TfidfVectorizer, class_labels) # Instanciate the model class
LR_Tfidf_model.fit() # Fitting the model
y_pred = LR_Tfidf_model.predict() # Make the predictions
class_labels=label_encoder.inverse_transform(range(8))

LR_Tfidf_model.report(y_pred,class_labels) # Show report on model performances

c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



               precision    recall  f1-score   support

     BUSINESS       0.79      0.78      0.78      2922
ENTERTAINMENT       0.83      0.88      0.85      2952
       HEALTH       0.75      0.81      0.78      3006
       NATION       0.66      0.70      0.68      2970
      SCIENCE       0.91      0.76      0.82       764
       SPORTS       0.92      0.92      0.92      3009
   TECHNOLOGY       0.89      0.86      0.87      3111
        WORLD       0.72      0.65      0.68      3021

     accuracy                           0.80     21755
    macro avg       0.81      0.79      0.80     21755
 weighted avg       0.80      0.80      0.80     21755



In [20]:
SGD_Tfidf_model = Model(filtered_X,y_enc, SGDClassifier(),TfidfVectorizer, class_labels) # Instanciate the model class
SGD_Tfidf_model.fit() # Fitting the model
y_pred = SGD_Tfidf_model.predict() # Make the predictions
class_labels=label_encoder.inverse_transform(range(8))

SGD_Tfidf_model.report(y_pred,class_labels) # Show report on model performances

               precision    recall  f1-score   support

     BUSINESS       0.79      0.75      0.77      2922
ENTERTAINMENT       0.81      0.88      0.84      2952
       HEALTH       0.69      0.84      0.76      3006
       NATION       0.71      0.60      0.65      2970
      SCIENCE       0.85      0.78      0.81       764
       SPORTS       0.86      0.92      0.89      3009
   TECHNOLOGY       0.85      0.86      0.85      3111
        WORLD       0.73      0.61      0.66      3021

     accuracy                           0.78     21755
    macro avg       0.79      0.78      0.78     21755
 weighted avg       0.78      0.78      0.78     21755

